# Demonstrations of Sifter Query Functionality

 - This notebook contains a mixture of demos of finished code, as well as WIP experimentation with ways to speed-up the code 

In [6]:
import time
import numpy as np
import scipy.stats as stats
import math
import random
from collections import defaultdict
import os
import sys
from collections import Counter
import glob 
import warnings
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import importlib
from astropy.time import Time

# -------------------------------------------------------------------------------------
# Local imports
# -------------------------------------------------------------------------------------
sys.path.append( os.path.join(os.path.dirname(os.getcwd() ), 'sifter')  )
sys.path.append( os.path.join( os.path.dirname(os.path.dirname(os.getcwd()) ), 'orbit_cheby', 'orbit_cheby')  )
import orbit_cheby
import query 
importlib.reload(orbit_cheby)
importlib.reload(query)


<module 'query' from '/Users/matthewjohnpayne/Envs/sifter/sifter/query.py'>

### Input Orbit Cheby Dictionary
 - We need a 'multi-sector-cheby-dictionary' containing cheby coeffs to represent an orbit
 - We'll use one created as part of the 'orbit_cheby" development work
 - The idea is that some earlier nbody integration produced output that was then approximated as chebyshev polynomials whose coefficients were saved in said dictionary

In [3]:
%%time 
importlib.reload(orbit_cheby)

# Set up a filepath (to file created as part of orbit_cheby development )
text_filepath = os.path.join( os.path.dirname(os.path.dirname(os.getcwd()) ), 'orbit_cheby', 'dev_data', '2022AA_demo.txt')

# Generate the dictionary by reading the (synthetic) nbody data from file and creating cheby-coeff-dict
multi_sector_cheby_dict = orbit_cheby.generate_multi_sector_cheby_dict_from_nbody_text( 
                                    text_filepath ,
                                    45000,
                                    55000, 
                                    minorder=17,
                                    CHECK = True )

print ( 'Number of sectors = ' , len(multi_sector_cheby_dict['sectors']) )

Number of sectors =  313
CPU times: user 846 ms, sys: 32.1 ms, total: 878 ms
Wall time: 881 ms


##### Observatory Positions 
- Some of these calculations require that the positions of the observatory are provided as inputs: I.e. I want to completely separate the obs-posn stuff from the cheby evaluation calculations
- For the sake of these demonstrations, where possible I am going to generate a set of standardized input observatory-positions (using a silly ~circular approximation) and then provide those as inputs. 
     

In [5]:
%%time 

# Create a sample of observatory-positions as inputs 
# - Note the decision to generate the obs posns in the ecliptic
# - The intent is to minimize the subsequent coordinate rotations 
#   (e..g not rotating from ecliptic-to-equatorial) 
#   when comparing the orbits to the observatory/observations
nSample = 1e4 
times = Time( 42000. + np.arange( nSample ) , format='mjd', scale='tdb')
r = 1.0 
XYZobservatory_heliocentric_ecliptic = np.array( [    r*np.cos(times.value), 
                                                      r*np.sin(times.value) , 
                                                  0.3*r*np.cos(times.value)]).T

print(XYZobservatory_heliocentric_ecliptic.shape)
print(XYZobservatory_heliocentric_ecliptic[0])

(10000, 3)
[-0.99885712 -0.04779594 -0.29965714]
CPU times: user 3.19 ms, sys: 2.16 ms, total: 5.36 ms
Wall time: 3.79 ms


## Demo toplevel "query" function
 - It is expected that a "standard" usage of the sifter routine will involve a single cal lto the "query" function
 - This will take an orbit_cheby_dictionaty as input
 - It will also require access to an observatory_position look-up function
 
N.B. This top-level query calls a number of underlying functions. 
 -  _check_query_inputs(  cheby_dict_for_orbit , param_dict )
 - _get_nightly_healpix( cheby_dict_for_orbit )   
 - _query_precalc(JD_list, HP_list)
 - _get_precise_matches( self.approx_dictionary , cheby_dict_for_orbit , param_dict )
 - get_results( list_of_tracklet_tuples_precise , param_dict )
    
These functions are examined in more detail lower-down in this notebook 
 

## Evaluate main components used within "query" function
 - The above "query" function clals a number of key components 
 - Here I examine/demonstrate/test some of those components 

######  _check _ query _ inputs
 - Simple routine to check whether the supplied inputs are correctly formatted / have the correct content 

In [ ]:
query._check_query_inputs(  cheby_dict_for_orbit , param_dict )

###### _ get _ nightly _ healpix (evaluates orbital positions on integer days)
 - Essentially a wrapper around the "cheby.generate_HP_from_cheby" function 
 - But note that in order to run automatically, I/we need to sort-out some means to provide access to the nightly positions of the observatories (don't need to worry about the positions at the times of hte tracklets as these will be saved within the sifter pre-calc structures) 

In [7]:
%%time 
importlib.reload(orbit_cheby)

# Get Nightly Healpix (evaluates orbital positions on integer days)
JD_list, HP_list = query._get_nightly_healpix( multi_sector_cheby_dict )
JD_list, HP_list

***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VAL

***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VAL

***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VAL

***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VAL

***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VAL

...

***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VAL

***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VAL

***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VAL

***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VALUES: NEED TO POPULATE CORRECTLY !!! ***
***DUMMY VAL

([40000,
  40001,
  40002,
  40003,
  40004,
  40005,
  40006,
  40007,
  40008,
  40009,
  40010,
  40011,
  40012,
  40013,
  40014,
  40015,
  40016,
  40017,
  40018,
  40019,
  40020,
  40021,
  40022,
  40023,
  40024,
  40025,
  40026,
  40027,
  40028,
  40029,
  40030,
  40031,
  40032,
  40033,
  40034,
  40035,
  40036,
  40037,
  40038,
  40039,
  40040,
  40041,
  40042,
  40043,
  40044,
  40045,
  40046,
  40047,
  40048,
  40049,
  40050,
  40051,
  40052,
  40053,
  40054,
  40055,
  40056,
  40057,
  40058,
  40059,
  40060,
  40061,
  40062,
  40063,
  40064,
  40065,
  40066,
  40067,
  40068,
  40069,
  40070,
  40071,
  40072,
  40073,
  40074,
  40075,
  40076,
  40077,
  40078,
  40079,
  40080,
  40081,
  40082,
  40083,
  40084,
  40085,
  40086,
  40087,
  40088,
  40089,
  40090,
  40091,
  40092,
  40093,
  40094,
  40095,
  40096,
  40097,
  40098,
  40099,
  40100,
  40101,
  40102,
  40103,
  40104,
  40105,
  40106,
  40107,
  40108,
  40109,
  40110,
 

###### _ query _ precalc 
 - Routine to quickly query an sqlite-db to understand whether there are any tracklets in a given set of (JD,HP) pairs  
 - It's job is to return approximate/possible matches for a given night, that will be subsequently refined by the "Get_precise_matches" function (see below) 

In [ ]:
_query_precalc(JD_list, HP_list)

###### _ get _ precise _ matches
 - Takes the approximate matches (from _query_precalc) and then does precise matching against tracklets
 - Note that the tracklets contain the pre-calculated observatory positions at the time of calculations
 
### THIS TO BE WRITTEN BY MIKE ALEXANDERSEN ??? ***

In [ ]:
_get_precise_matches

###### get _ results

In [ ]:
get_results